In [42]:
import pandas as pd
import numpy as np
# Load data
df = pd.read_csv("../data/raw/online_retail.csv")
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [43]:
# Check size of Data: row, column, datatype
df.shape
# df.columns
# df.dtypes

(541909, 8)

In [44]:
# Review data for overview  
df.info()

# Trả lời các câu hỏi quan trọng:

# Cột nào bị thiếu dữ liệu?
# Kiểu dữ liệu có đúng không?
# Có cột nào đáng nghi?

# 📌 Với dataset này, bạn sẽ thấy:

# CustomerID bị thiếu
# InvoiceDate đang là object (string)
# Quantity, UnitPrice là số

<class 'pandas.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  str    
 1   StockCode    541909 non-null  str    
 2   Description  540455 non-null  str    
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  str    
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  str    
dtypes: float64(2), int64(1), str(5)
memory usage: 33.1 MB


## Initial observations
- CustomerID contains missing values
- InvoiceDate is object, should be datetime
- Dataset represents transactional sales data

In [45]:
# Check Missing values
df.isna().sum()
# df.isnull().sum()


# CustomerID thiếu khá nhiều
# Các cột khác gần như đầy đủ

# 👉 Câu hỏi kỹ thuật phải đặt ra:
# Missing CustomerID có drop được không?
# Hay đó là khách vãng lai?

# 📌 Trong retail:
# Missing CustomerID = khách không đăng ký → không phải lỗi

# ➡️ EDA giúp bạn KHÔNG drop nhầm data

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## Missing data decision
- CustomerID missing values likely represent guest customers
=> Will keep rows with missing CustomerID

In [46]:
# Check critical data
df.describe()


# Quantity có giá trị âm ❗
# UnitPrice có giá trị = 0 ❗

# 📌 Ý nghĩa nghiệp vụ:
# Quantity < 0 → hàng trả lại (refund)
# UnitPrice = 0 → khuyến mãi / lỗi dữ liệu

# 👉 EDA giúp hiểu business logic, không chỉ code

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


## Anomalies observed
- Negative Quantity indicates returned items
- Zero UnitPrice may indicate promotions or data issues

In [47]:
#Check text 
df.select_dtypes(include="object").head()


# 👉 Kiểm tra:

# Description có bị null không
# Country có consistent không
# InvoiceNo có dạng đặc biệt không

/var/folders/93/1gpxyqm14c71cxccb_1_5tmm0000gn/T/ipykernel_82630/2624877839.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  df.select_dtypes(include="object").head()


,InvoiceNo,StockCode,Description,InvoiceDate,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,2010-12-01 08:26:00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,United Kingdom


In [48]:
# Check special Invoice
df[df["InvoiceNo"].str.startswith("C", na=False)].head()

# Kết luận:

# Invoice bắt đầu bằng "C" → cancel / credit invoice
# 👉 EDA giúp phát hiện rule ẩn trong data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [49]:
# Create feature
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]
df[["Quantity", "UnitPrice", "TotalPrice"]].head()

# 👉 Đây là EDA feature, chưa phải code production.

,Quantity,UnitPrice,TotalPrice
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34


In [50]:
#Check adnormal Total price
df["TotalPrice"].describe()

# 📌 Bạn sẽ thấy:

# TotalPrice âm → refund
# Max rất lớn → cần kiểm tra

count    541909.000000
mean         17.987795
std         378.810824
min     -168469.600000
25%           3.400000
50%           9.750000
75%          17.400000
max      168469.600000
Name: TotalPrice, dtype: float64

In [51]:
# Check and correct the time format
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Create details time feature
df["Year"] = df["InvoiceDate"].dt.year
df["Month"] = df["InvoiceDate"].dt.month
df["Hour"] = df["InvoiceDate"].dt.hour

## EDA Summary

- Dataset contains ~541k retail transactions
- Missing CustomerID represents guest customers
- Negative Quantity and TotalPrice indicate returns
- Invoice numbers starting with "C" indicate cancellations
- Created TotalPrice feature for sales analysis
- Extracted time-based features from InvoiceDate